<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re
from function_library import *
from function_library2 import *

In [53]:
re_behalf = re.compile(r'(.*)[Oo]n [Bb]ehalf')
re_bracket = re.compile(r'(.*)[\[\<\(]mailto\:(.*)[\]\>\)]')
re_bracket2 = re.compile(r'(.*)[\[\<\(](.*)[\]\>\)]')

re_name1 = re.compile(r'.*?([A-Za-z]+\s?[A-Z]?\.?)\s([A-Za-z]+)')
re_name2 = re.compile(r'.*?([A-Za-z]+),\s([A-Za-z]+\s?[A-Z]?\.?)')
re_email = re.compile(r'.*?([a-zA-Z_]*\.?\w+@[a-zA-Z_0-9]*\.?[a-zA-Z_]*\.?[a-zA-Z]{2,3})')


In [54]:
expr = re.compile(r'(.*)[\[\<\(]mailto\:(.*)[\]\>\)]')

In [82]:
# either the sender or the recipient
# df = pd.read_csv('new_clean_output.csv',index_col = 0)
#df = pd.read_csv('output_0211.csv')
#df = pd.read_csv('old_output.csv')   #.iloc[0:1,:]
df = pd.read_csv('o.csv')

# df = df.drop_duplicates(keep='first',subset=['Sent'])
# df = df.reset_index(drop=True)
# print(df.head())
print(df['From'])

from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()
print("==> .values.tolist():from_list: ", from_list)
print("df['From'].values: ", df['From'].values)
print("eval(df.From): ", eval("df['From'].values"))

0    ['Shane A. Moniz']
Name: From, dtype: object
==> .values.tolist():from_list:  ["['Shane A. Moniz']"]
df['From'].values:  ["['Shane A. Moniz']"]
eval(df.From):  ["['Shane A. Moniz']"]


In [83]:
from_list.append("gordon")
lst = eval(from_list[0])[0]
from_list = lst
from_list

'Shane A. Moniz'

In [84]:
df # GE

,From,Sent,To,CC,Subject,Attachments,Importance,Body,isDisplacement,isThread,isAutoMessage,hasAllCapLine,hasBadDate
0,['Shane A. Moniz'],"['Friday, May 08, 2015 11:38 AM', 'March 2, 20...","['Brooks Hayes (brooks@culpeppercc.com)', 'Sha...",['Ryan Grindler (rgrindler@101tally.com); Rose...,"[""Edison RFI's""]",['RFI 008 Response.pdf; RFI 042 Response.pdf; ...,[],"['Response.pdf', 'Brooks, attached is our resp...",False,False,False,False,True


In [57]:
people_list = []
named_email_list = []
name_list = []
# find unique complete person in From section

dct = {}

for f in from_list: 
    if pd.isnull(f):
        continue
    f = f.lower()
    dct['f'] = f
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
        dct['f1'] = f
    if re_bracket.match(f):
        dct['re_bracket.match'] = True
        tname, temail = re_bracket.findall(f)[0]
        tname=tname.lower().strip()
        temail=temail.lower()
        dct['temail'] = temail
        if re_email.match(temail) and re_email.match(tname) == None:
            email = re_email.findall(temail)[0]
            if len(tname.split()) != 2:
                first_name = tname
                last_name = ' '
            else:
                if re_name1.match(tname):
                    name = re_name1.findall(tname)[0]
                    first_name = name[0]
                    last_name = name[1]
                elif re_name2.match(tname): 
                    name = re_name2.findall(tname)[0]
                    first_name = name[1]
                    last_name = name[0]
                else:
                    print("email: ", email)
                    print("-----------------------------------")
                    first_name = 'unrognized name'
                    last_name = tname
                    print('error: unrognized name:',tname)
                    
            if email not in named_email_list:
                person = (first_name, last_name, email)
                people_list.append(person)
                named_email_list.append(email)
                name_list.append(first_name + ' ' + last_name)
           

In [58]:
len(people_list)

0

In [41]:
to_list

["['Brooks Hayes (brooks@culpeppercc.com)', 'Shane Moniz']"]

In [59]:
# find unique complete person in TO section
for ts in to_list: 
    if pd.isnull(ts):
        continue
    ts = ts.lower()
    ts = ts.split(';')
    for t in ts:
        t = t.strip("'")
        if re_bracket.match(t) or re_bracket2.match(t):
            if re_bracket.match(t):
                tname, temail = re_bracket.findall(t)[0]
            else:
                tname, temail = re_bracket2.findall(t)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        first_name = 'unrognized name'
                        last_name = tname
                        print('error: unrognized name:',tname)
                        
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                   

In [60]:
len(people_list), people_list

(1, [('brooks', 'hayes', 'brooks@culpeppercc.com')])

In [44]:
# find unique complete person in CC section
for ccs in cc_list: 
    if pd.isnull(ccs):
        continue
    ccs = ccs.lower()
    ccs = ccs.split(';')
    for cc in ccs:
        cc = cc.strip("'")
        if re_bracket.match(cc) or re_bracket2.match(cc):
            if re_bracket.match(cc):
                tname, temail = re_bracket.findall(cc)[0]
            else:
                tname, temail = re_bracket2.findall(cc)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        first_name = 'unrognized name'
                        last_name = tname
                        print('error: unrognized name:',tname)
                        
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
             

In [45]:
len(people_list), people_list

(2,
 [('brooks', 'hayes', 'brooks@culpeppercc.com'),
  ('ryan', 'grindler', 'rgrindler@101tally.com')])

In [46]:
new_from_list = []

unknown_idx = 0
# replace the From section with unique people information

for f in from_list: 
    if pd.isnull(f):
        print('null')
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), 'f'+str(unknown_idx)+'_'+'l'+str(unknown_idx))
        unknown_idx = unknown_idx + 1
        new_from_list.append(person)
        continue
    email_exist_flag = 0
    f = f.lower().strip("'")
    email = ''
    first_name = ''
    last_name = ''
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
        print("f1: f: ", f)
        
    if re_email.match(f):
        print("if 0, f:  ", f)
        email = re_email.findall(f)[0]
        email = email.lower()
        email_exist_flag = 1
    if len(f.split()) != 2:
        print("if 1, f: ", f)
        first_name = f
        last_name = ' '
    else:
        print("else")
        if re_name1.match(f):
            name = re_name1.findall(f)[0]
            first_name = name[0]
            last_name = name[1]
        elif re_name2.match(f): 
            name = re_name2.findall(f)[0]
            first_name = name[1]
            last_name = name[0]
        elif email_exist_flag == 1:
            pass
        else:
            print('error: cannot find name and email in f')
            print('f:', f)
            first_name = 'fake'
            last_name = f
            
    name = first_name + ' ' + last_name
    print("name: ", name)
    print("named_email_list: ", named_email_list)
    print("email: ", email)

    
    if email in named_email_list:
        idx = named_email_list.index(email)
        new_from_list.append(people_list[idx])
    elif name in name_list:
        print("elif 1, name: ", name, ",  name_list: ", name_list)
        idx = name_list.index(name)
        new_from_list.append(people_list[idx])
    elif email_exist_flag == 1:
        print("elif 2, email_exist_flag: ", email_exist_flag)
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
        new_from_list.append(person)
        unknown_idx = unknown_idx + 1
    else:
        print("else")
        person = (first_name, last_name, first_name + '_' + last_name)
        # new_from_list.append(person)   # original 
        new_from_list = person
        print("new_from_list: ", new_from_list)

if 1, f:  ['shane a. moniz']
name:  ['shane a. moniz']  
named_email_list:  ['brooks@culpeppercc.com', 'rgrindler@101tally.com']
email:  
else
new_from_list:  ("['shane a. moniz']", ' ', "['shane a. moniz']_ ")


In [47]:
len(new_from_list), new_from_list

(3, ("['shane a. moniz']", ' ', "['shane a. moniz']_ "))

In [133]:
len(from_list)

71143

In [134]:
new_to_lists = []

# replace the To section with unique people information

for ts in to_list: 
    if pd.isnull(ts):
        new_to_lists.append([])
        continue
    ts = ts.lower()
    ts = ts.split(';')
    new_to_list = []
    for t in ts:
        t = t.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(t):
            t = re_behalf.findall(t)[0]

        if re_email.match(t):
            email = re_email.findall(t)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(t.split()) != 2:
            first_name = t
            last_name = ' '
        else:
            if re_name1.match(t):
                name = re_name1.findall(t)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(t): 
                name = re_name2.findall(t)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = t
                print('error: cannot find name and email in t, make fake name')
                print('t:', t)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_to_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_to_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_to_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_to_list.append(person)
    new_to_lists.append(new_to_list)

error: cannot find name and email in t, make fake name
t: [' scof']
error: cannot find name and email in t, make fake name
t: [' announcements
error: cannot find name and email in t, make fake name
t: [' undisclosed-recipients:
error: cannot find name and email in t, make fake name
t: [' zprgroup']
error: cannot find name and email in t, make fake name
t: [' zprgroup']
error: cannot find name and email in t, make fake name
t: [' announcements
error: cannot find name and email in t, make fake name
t: [' undisclosed-recipients:
error: cannot find name and email in t, make fake name
t:  andersonjr., fred
error: cannot find name and email in t, make fake name
t: [' zprgroup']
error: cannot find name and email in t, make fake name
t:  impact100 grants
error: cannot find name and email in t, make fake name
t:  jeﬀ price
error: cannot find name and email in t, make fake name
t: [' undisclosed-recipients:
error: cannot find name and email in t, make fake name
t: [' zprgroup']
error: cannot fin

In [135]:
len(to_list)

71143

In [136]:
len(new_to_lists)

71143

In [137]:
new_to_lists[:10]

[[('brooks', 'hayes', 'brooks@culpeppercc.com')],
 [('kristen', 'coons', 'kristen.coons@talgov.com')],
 [('kim', 'rivers', 'kim@inkbridge.com')],
 [('alison', 'faris', 'alison.faris@talgov.com')],
 [('andrew', 'gillum', 'andrew.gillum@talgov.com')],
 [('allen', 'thompson', 'downtownmarket@earthlink.net')],
 [('michael', 'alfano', 'michael_alfano')],
 [('vince', 'long', 'longv@leoncounty')],
 [('chad', 'kittrell', 'chad@hunterandharp.com')],
 [('sue', 'dick', 'sdick@talchamber.com'),
  ('dana', 'noles', 'dnoles@talchamber.com'),
  ('whitney', 'weeks', 'whitney_weeks'),
  (' ed edward murray jr.', ' ', ' ed edward murray jr._ '),
  ('kristin', 'dozier', 'dozierk@leoncountyfl.gov'),
  ('cecilia', 'homison', 'chomison@firstcommercecu.org'),
  ('lonnie', 'ballard', 'lonnie.ballard@talgov.com'),
  ('scott', 'balog', 'balogs@tcc.fl.edu'),
  ('kathy', 'bell', 'kgb@coloneybell.com'),
  ('reggie l. bouthillier', ' ', 'rbouthillier@stearnsweaver.com'),
  ('jt', 'burnette', 'jt@inkbridge.com'),
  

In [138]:
new_cc_lists = []
# replace the cc section with unique people information

for ccs in cc_list: 
    if pd.isnull(ccs):
        new_cc_lists.append([])
        continue
    ccs = ccs.lower()
    ccs = ccs.split(';')
    new_cc_list = []
    if len(ccs) == 1 and ccs[0].strip() == '':
        new_cc_lists.append(new_cc_list)
        continue
    for cc in ccs:
        cc = cc.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(cc):
            cc = re_behalf.findall(cc)[0]

        if re_email.match(cc):
            email = re_email.findall(cc)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(cc.split()) != 2:
            first_name = cc
            last_name = ' '
        else:
            if re_name1.match(cc):
                name = re_name1.findall(cc)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(cc): 
                name = re_name2.findall(cc)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = cc
                print('error: cannot find name and email in cc, make fake name')
                print('cc:', cc)

        name = first_name + ' ' + last_name

        if email in named_email_list:
            idx = named_email_list.index(email)
            new_cc_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_cc_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_cc_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_cc_list.append(person)
    new_cc_lists.append(new_cc_list)

error: cannot find name and email in cc, make fake name
cc:  maisbel(mae) mendez']
error: cannot find name and email in cc, make fake name
cc: [' charles,']
error: cannot find name and email in cc, make fake name
cc: [' charles,']
error: cannot find name and email in cc, make fake name
cc:  maisbel(mae) mendez
error: cannot find name and email in cc, make fake name
cc: [' edwar']
error: cannot find name and email in cc, make fake name
cc:  ramon.alexander@ myfloridahouse.gov
error: cannot find name and email in cc, make fake name
cc: [' gary']
error: cannot find name and email in cc, make fake name
cc: [' ep&er']
error: cannot find name and email in cc, make fake name
cc: [' josh/breck,']
error: cannot find name and email in cc, make fake name
cc: ["adam' 'corey
error: cannot find name and email in cc, make fake name
cc: [' april
error: cannot find name and email in cc, make fake name
cc: [' charles,']
error: cannot find name and email in cc, make fake name
cc:  'fastsigns 373
error: c

In [139]:
len(cc_list)

71143

In [140]:
len(new_cc_lists)

71143

In [141]:
new_cc_lists[:10]

[[('ryan', 'grindler', 'rgrindler@101tally.com'),
  ('david', 'rosenfeld', 'david.rosenfeld@talgov.com')],
 [('patrick', 'hurley', 'patrick.hurley@talgov.com')],
 [('jon', 'brown', 'brownjon@leoncountyfl.gov'),
  ('mathieu', 'cavell', 'cavellm@leoncountyfl.gov'),
  ('team', 'cmr', 'team_cmr'),
  ('victoria', 'connell', 'victoria_connell'),
  ('deborah', 'craig', 'deborah.craig@talgov.com'),
  ('stephanie', 'holloway', 'holloways@leoncountyfl.gov'),
  ('lindsay', 'jordan', 'lindsay_jordan'),
  ('angeline', 'taylor', 'angeline_taylor'),
  ('jay', 'townsend', 'jltownsend1986@gmail.com'),
  (" jessicamiller']", ' ', " jessicamiller']_ ")],
 [('paige', 'carter', 'paige.tallahasseedowntown@gmail.com'),
  (' parade springtime tallahassee', ' ', ' parade springtime tallahassee_ '),
  ('edward', 'kring', 'edward.kring@talgov.com')],
 [('f70017', 'l70017', 'alan.williams@my')],
 [('[]', ' ', '[]_ ')],
 [('dustin', 'daniels', 'r.daniels@gmail.com'),
  (" simpson, roxanne m']", ' ', " simpson, rox

In [142]:
df_new = df.copy()
df_new['From'] = new_from_list
df_new['To'] = new_to_lists
df_new['CC'] = new_cc_lists
df_new


,From,Sent,To,CC,Subject,Attachments,Importance,Body,isDisplacement,isThread,isAutoMessage,hasAllCapLine,hasBadDate
0,"(['shane a. moniz'], , ['shane a. moniz']_ )","['Friday, May 08, 2015 11:38 AM', 'March 2, 20...","[(brooks, hayes, brooks@culpeppercc.com)]","[(ryan, grindler, rgrindler@101tally.com), (da...","[""Edison RFI's""]",['RFI 008 Response.pdf; RFI 042 Response.pdf; ...,[],"['Response.pdf', 'Brooks, attached is our resp...",False,False,False,False,True
1,"(mark, beaudoin, mark.beaudoin@talgov.com)","['Tuesday, August 19, 2014 5:21 PM']","[(kristen, coons, kristen.coons@talgov.com)]","[(patrick, hurley, patrick.hurley@talgov.com)]",['RE: Revised lease'],[],[],['Good by me!'],False,False,False,False,False
2,"(ken, morris, morrisk@leoncountyfl.gov)","['Tuesday, May 28, 2013 1:22 PM']","[(kim, rivers, kim@inkbridge.com)]","[(jon, brown, brownjon@leoncountyfl.gov), (mat...",['Proposed Imagine Schedule'],[],[],"['Kim,', 'See comments in blue regarding the p...",False,False,False,False,False
3,"(jennifer, naff, director@springtimetallahasse...","['Wednesday, March 15, 2017 5:32 PM']","[(alison, faris, alison.faris@talgov.com)]","[(paige, carter, paige.tallahasseedowntown@gma...",['RE: Springtime Parade'],[],[],"['Hi Alison,', 'Dont apologize, I know its a l...",False,False,False,True,False
4,"(paul, lamaster, paullamaster250@comcast.net)","['Sunday, June 7, 2015 at 1:33:39 PM Eastern D...","[(andrew, gillum, andrew.gillum@talgov.com)]","[(f70017, l70017, alan.williams@my)]",['Tony Carvajal: Develop strategy to get ahead...,[],[],['Highly recommend you take a moment out of yo...,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71138,"(paul, lamaster, paullamaster250@comcast.net)","['Thursday, August 10, 2017 at 11:35:27 AM Eas...","[(andrew, gillum, andrew.gillum@talgov.com)]","[([], , []_ )]",['Tallahassee Democrat E-Edi4on Ar4cle'],[],[],"['nancy.miller@talgov.com, scoS.maddox@talgov....",True,False,False,False,False
71139,"(ryan, grindler, ryan@edisontally.com)","['Wednesday, February 22, 2017 at 6:27:34 PM E...","[([' paige carter-smith'], , [' paige carter-...","[([' steve adams, ashley edwards, allison scor...",['Re: Summer Movie Night Series Discussion Mee...,[],[],"['All,', 'We are s-ll working on some logis-ca...",True,False,False,False,False
71140,"(allie, fleming, allison.fleming@talgov.com)","[' Tuesday, May 28, 2013 2:38 PM']","[(scott, maddox, scott@scottmaddox.com)]","[(paige, carter, paigecartersmith@gmail.com)]",[' Call Sheet'],[' Call List 5-28.xlsx'],[],['Current Call sheet attached - including this...,True,False,False,False,False
71141,"(scott, williams, scott@hunterandharp.com)","['Monday, April 27, 2015 5:24 PM']","[(scott, rowse, scott@morethanbuildings.com)]","[([], , []_ )]",['FW: Gateway Sprinklers and Sidewalks'],[],[],"['Do you have a contact for the sprinklers?', ...",False,False,False,False,False


In [143]:
df_new.to_csv('output_0211_name_standardized.csv', index=0)

In [147]:
def print_df(row_nb):
    print(df.iloc[row_nb,:])
    print("---------------------------------------------------------------------")
    print(df_new.iloc[row_nb,:])

In [148]:
print_df(0)

From                                             ['Shane A. Moniz']
Sent              ['Friday, May 08, 2015 11:38 AM', 'March 2, 20...
To                ['Brooks Hayes (brooks@culpeppercc.com)', 'Sha...
CC                ['Ryan Grindler (rgrindler@101tally.com); Rose...
Subject                                            ["Edison RFI's"]
Attachments       ['RFI 008 Response.pdf; RFI 042 Response.pdf; ...
Importance                                                       []
Body              ['Response.pdf', 'Brooks, attached is our resp...
isDisplacement                                                False
isThread                                                      False
isAutoMessage                                                 False
hasAllCapLine                                                 False
hasBadDate                                                     True
Name: 0, dtype: object
---------------------------------------------------------------------
From                  (

In [146]:
df_new.iloc[933,:]

From              (larry, schmidt, larry@produc.onsupportgroup.com)
Sent                               ['Tuesday, May 9, 2017 5:26 PM']
To                    [(jessica, brown, jessicae.brown@talgov.com)]
CC                                                  [([],  , []_ )]
Subject           ['RE: Couple of Items in Prepara.on for Saturd...
Attachments                                                      []
Importance                                                       []
Body              ['Start info owing both ways. I will need thei...
isDisplacement                                                False
isThread                                                      False
isAutoMessage                                                 False
hasAllCapLine                                                 False
hasBadDate                                                    False
Name: 933, dtype: object